In [1]:
"""
STEP 1

There is the customer_master.xlsx file as a definition table for data preparation. 
It helps to exlude wrong inforamtion in an internal data source which cannot be corrected.

columns: ['Customer', 'Tier', 'Accounts', 'Company', 'the_newest_OPCO',
       'Customer Name', 'ONE_NAME', 'Indirect/Direct', 'Channel3', 'New Type4']

There is a budget file which budget information for two fiscal years. 

columns: ['sold_to_customer', 'sold_to_customer_n', 'Country', 'Country Name',
       'City', 'Type', 'OI Bgt FY21', 'OI Bgt FY22', 'Comments']

Main goal of this code to combine these two files in one which would be used as one of 
Tableau data sources.

"""

"\nSTEP 1\n\nThere is the customer_master.xlsx file as a definition table for data preparation. \nIt helps to exlude wrong inforamtion in an internal data source which cannot be corrected.\n\ncolumns: ['Customer', 'Tier', 'Accounts', 'Company', 'the_newest_OPCO',\n       'Customer Name', 'ONE_NAME', 'Indirect/Direct', 'Channel3', 'New Type4']\n\nThere is a budget file which budget information for two fiscal years. \n\ncolumns: ['sold_to_customer', 'sold_to_customer_n', 'Country', 'Country Name',\n       'City', 'Type', 'OI Bgt FY21', 'OI Bgt FY22', 'Comments']\n\nMain goal of this code to combine these two files in one which would be used as one of \nTableau data sources.\n\n"

In [32]:
import pandas as pd
import numpy as np

#budget df preparation
budget = pd.read_excel('data_files/initial_budget_file.xlsx', sheet_name='Budget')
budget['sold_to_customer'] = budget['sold_to_customer'].astype('str')
budget.sold_to_customer = "0" + budget.sold_to_customer

#budget_df = pd.read_excel('data_files/budget_file.xlsx', sheet_name='budget_by_month', index_col='sold_to_customer')
#budget_df = budget_df.reset_index()
#budget_df.sold_to_customer = budget_df.sold_to_customer.astype('str') # the internal database has ID beginning with 0 in str format
#budget_df.sold_to_customer = "0" + budget_df.sold_to_customer

#customer master df preparation
customer_df = pd.read_excel('data_files/customer_master.xlsx', sheet_name='data')
customer_df = customer_df.loc[:,['Customer', 'Tier', 'the_newest_OPCO',
       'Customer Name', 'ONE_NAME', 'Indirect/Direct', 'Channel3', 'New Type4']]
customer_df = customer_df.rename(columns={'Customer':'sold_to_customer','the_newest_OPCO':'OPCO', 'Channel3':'Channel', 'New Type4':'Type'})
customer_df.sold_to_customer = customer_df.sold_to_customer.astype('str') # the internal database has ID beginning with 0 in str format
customer_df.sold_to_customer = "0" + customer_df.sold_to_customer
customer_df=customer_df.dropna(subset=['ONE_NAME'])
customer_df = customer_df.reset_index()
#customer_df = customer_df.set_index('sold_to_customer')

In [33]:
class One_name(object):

    """Combining different ID and Company Names in One_name with all necessary parametres"""

    def __init__(self, one_name):
        self.one_name = one_name
        self.sold_to_customer = []
        self.sold_to_customer_n = []
        self.typ = ['Direct','Direct','End User']
        self.target = [0,0]

  
    def add_id(self, sold_to_customer):
        if sold_to_customer not in self.sold_to_customer:
            self.sold_to_customer.append(sold_to_customer)
    
    def add_name(self, sold_to_customer_n):
        if sold_to_customer_n not in self.sold_to_customer_n:
            self.sold_to_customer_n.append(sold_to_customer_n)
    
    def add_type(self, typ):
        self.typ = typ
        
    def add_tier(self, tier):
        self.tier = tier    
    
    def add_opco(self, opco):
        self.opco = opco
        
    def add_target(self, target):
        if target[0] > 0:
            self.target[0] = self.target[0] + target[0]
        if target[1] > 0:
            self.target[1] = self.target[1] + target[1]
           
    def get_one(self):
        return self.one_name
    def get_id(self):
        return self.sold_to_customer
    def get_tier(self):
        return self.tier
    def get_opco(self):
        return self.opco
    def get_name(self):
        return self.sold_to_customer_n
    def get_typ(self):
        return self.typ
    def get_target(self):
        return self.target

In [34]:
check = []
comps = []
for i in range(len(customer_df)):
    p = One_name(customer_df.loc[i,'ONE_NAME'])
    if customer_df.loc[i,'ONE_NAME'] not in check:
        comps.append(p)  
    check.append(customer_df.loc[i,'ONE_NAME'])

for comp in comps:
    for i in range(len(customer_df)):
        if comp.get_one() == customer_df.loc[i,'ONE_NAME']:
            comp.add_id(customer_df.loc[i,'sold_to_customer'])
            comp.add_tier(customer_df.loc[i,'Tier'])
            comp.add_opco(customer_df.loc[i,'OPCO'])
            comp.add_name(customer_df.loc[i,'Customer Name'])
            comp.add_type([customer_df.loc[i,'Indirect/Direct'],customer_df.loc[i,'Channel'],customer_df.loc[i,'Type']])

for comp in comps:
    for i in range(len(budget)):
        if budget.loc[i,'sold_to_customer'] in comp.get_id() : 
            comp.add_name(budget.loc[i,'sold_to_customer_n'])
            comp.add_target([budget.loc[i,'OI Bgt FY21'], budget.loc[i,'OI Bgt FY22']])

In [35]:
final = pd.DataFrame(columns=['ID','OPCO', 'Tier', 'ONE_NAME', 'Indirect/Direct', 'Channel', 'Type', 'OI Bgt FY21', 'OI Bgt FY22'])

for comp in comps:
    for i in range(len(comp.get_id())):
        temp = pd.DataFrame(columns=['ID','OPCO', 'Tier', 'ONE_NAME', 'Indirect/Direct', 'Channel', 'Type', 'OI Bgt FY21', 'OI Bgt FY22'])
        temp.loc[i, 'ID'] = comp.get_id()[i]
        temp.loc[i, 'OPCO'] = comp.get_opco()
        temp.loc[i, 'Tier'] = comp.get_tier()
        temp.loc[i, 'ONE_NAME'] = comp.get_one()
        temp.loc[i, 'Indirect/Direct'] = comp.get_typ()[0]
        temp.loc[i, 'Channel'] = comp.get_typ()[1]
        temp.loc[i, 'Type'] = comp.get_typ()[2]
        if i == 0:
            temp.loc[i, 'OI Bgt FY21'] = comp.get_target()[0]
            temp.loc[i, 'OI Bgt FY22'] = comp.get_target()[1]
        else:
            temp.loc[i, 'OI Bgt FY21'] = 0
            temp.loc[i, 'OI Bgt FY22'] = 0
        
        final = pd.concat([final, temp], ignore_index=True)

In [36]:
writer = pd.ExcelWriter('data_files/budget_file_intermediate.xlsx', engine='xlsxwriter')
final.to_excel(writer, sheet_name='budget_by_month')
writer.save()